In [19]:
import pandas as pd
import numpy as np
from sql_functions import *
import plotly.express as px
import json
from normal_round import normal_round as normal_round

In [20]:
pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

In [21]:
df_laender_abs = get_dataframe('SELECT * FROM public.laender_jugend_tat_absolut_2022_until_2018;')
df_laender_rel = get_dataframe('SELECT * FROM public.laender_jugend_tat_relativ_2022_until_2018;')

In [22]:
df_laender_abs.head(5)

,schluessel,straftat,bundesland,sexus,tatverdaechtige_insgesamt,kinder_10_bis_unter_12,kinder_12_bis_unter_14,kinder_10_bis_unter_14,kinder_unter_14,jugendliche_14_bis_unter_16,jugendliche_16_bis_unter_18,jugendliche_14<18,heranwachsende_18_bis_unter_21,jugendl_u_heranwachsende_14_bis_unter_21,erwachsene_ab_21,year
0,------,Straftaten insgesamt,Baden-Württemberg,M,184801,1305,2975,4280,5246,6585,10078,16663,20122,36785,142770,2018
1,------,Straftaten insgesamt,Bayern,M,226208,1390,3101,4491,6276,6786,10604,17390,24065,41455,178477,2018
2,------,Straftaten insgesamt,Berlin,M,101377,831,1561,2392,3112,2881,3977,6858,7941,14799,83466,2018
3,------,Straftaten insgesamt,Brandenburg,M,49670,547,962,1509,2050,1869,2469,4338,3910,8248,39372,2018
4,------,Straftaten insgesamt,Bremen,M,18519,95,224,319,389,642,1125,1767,1934,3701,14429,2018


In [23]:
#df_laender_rel.tail()

### Calculate most occuring crime per state - rank and percentage of total

In [24]:
df_1 = df_laender_abs.copy()
#df_1 = df_1[df_1['sexus'] == 'X']
df_1 = df_1[['schluessel', 'straftat', 'bundesland', 'sexus', 'year', 'heranwachsende_18_bis_unter_21']]
df_1 = df_1.rename(columns={'heranwachsende_18_bis_unter_21': 'offenders'})

df_2 = df_1[df_1['schluessel'] == '------']
df_2 = df_2[['bundesland', 'sexus', 'year', 'offenders']]
df_laender_straftaten_insg_absolut = df_2.copy() # needed later

df_3 = df_1[df_1['schluessel'] != '------']

df_3['rank'] = (
    df_3.groupby(['bundesland', 'sexus', 'year'])['offenders']
        .rank(ascending=False, method='first')
        .astype(int)
)

df_4 = df_3.merge(df_2, on=['bundesland', 'sexus', 'year'], how='left')

df_4['certain_crime_percent_of_total_crime'] = df_4['offenders_x'] / df_4['offenders_y'] * 100
df_4 = df_4.drop(columns='offenders_y')
df_4 = df_4.reset_index()
df_4 = df_4.drop(columns='index')

values = df_4['certain_crime_percent_of_total_crime'].tolist()
new_values = []
for v in values:
    new_v = normal_round(num=v, ndigits=2)
    new_values.append(new_v)
    
df_4['certain_crime_percent_of_total_crime'] = new_values

df_4 = df_4[['schluessel', 'bundesland', 'sexus', 'year', 'straftat', 'rank', 'certain_crime_percent_of_total_crime']]

crime_rank_and_percent_of_total_per_state = df_4.copy()
crime_rank_and_percent_of_total_per_state.head()

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_12634/3493380453.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['rank'] = (


,schluessel,bundesland,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
0,100000,Baden-Württemberg,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,2.43
1,100000,Bayern,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,2.40
2,100000,Berlin,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6,2.07
3,100000,Brandenburg,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,3.48
4,100000,Bremen,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6,1.91


### Create overview - number of offenders per state - absolut and relativ

In [25]:
df_10 = df_laender_rel.copy()
df_10 = df_10.rename(columns={'heranwachsende_18_bis_unter_21_rel': 'offenders_rel'})
df_10 = df_10[(df_10['schluessel'] == '------')]

jugendkrim_insg_rel = df_10['offenders_rel'].tolist()
jugendkrim_insg_rel_rounded = []
for v in jugendkrim_insg_rel:
    v_new = int(normal_round(v, 0))
    jugendkrim_insg_rel_rounded.append(v_new)
df_10['offenders_rel'] = jugendkrim_insg_rel_rounded

df_10 = df_10[['bundesland', 'sexus', 'year', 'offenders_rel']]
df_10 = df_10.reset_index()
df_10 = df_10.drop(columns='index')

df_11 = df_laender_straftaten_insg_absolut.merge(df_10, on=['bundesland', 'sexus', 'year'], how='left')
df_laender_straftaten_insg_abs_u_rel = df_11.copy()
df_laender_straftaten_insg_abs_u_rel.head()

,bundesland,sexus,year,offenders,offenders_rel
0,Baden-Württemberg,M,2018,20122,9909
1,Bayern,M,2018,24065,10521
2,Berlin,M,2018,7941,15860
3,Brandenburg,M,2018,3910,12308
4,Bremen,M,2018,1934,15209


### Merge crimes that <br>rank on 1, 2, and 3 <br>and <br>overall number of crimes - absolut and relativ <br>to every state for every year (->one DataFrame)

In [26]:
df_20 = crime_rank_and_percent_of_total_per_state.copy()
df_rank_1 = df_20[df_20['rank'] == 1]
df_rank_2 = df_20[df_20['rank'] == 2]
df_rank_3 = df_20[df_20['rank'] == 3]
df_rank_4 = df_20[df_20['rank'] == 4]
df_rank_5 = df_20[df_20['rank'] == 5]
df_rank_1.head()

,schluessel,bundesland,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
98,220000,Berlin,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,21.03
100,220000,Bremen,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,22.96
105,220000,Nordrhein-Westfalen,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,26.84
107,220000,Saarland,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,24.28
110,220000,Schleswig-Holstein,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,24.05


In [27]:
df_30 = df_laender_straftaten_insg_abs_u_rel.copy()

df_31 = df_30.merge(df_rank_1, on=['bundesland', 'sexus', 'year'], how='left')
df_31 = df_31.rename(columns={'straftat': 'crimes_on_rank_1',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_1_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_1'})
df_31 = df_31.drop(columns='rank')

df_32 = df_31.merge(df_rank_2, on=['bundesland', 'sexus', 'year'], how='left')
df_32 = df_32.rename(columns={'straftat': 'crimes_on_rank_2',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_2_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_2'})
df_32 = df_32.drop(columns='rank')

df_33 = df_32.merge(df_rank_3, on=['bundesland', 'sexus', 'year'], how='left')
df_33 = df_33.rename(columns={'straftat': 'crimes_on_rank_3',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_3_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_3'})
df_33 = df_33.drop(columns='rank')

df_34 = df_33.merge(df_rank_4, on=['bundesland', 'sexus', 'year'], how='left')
df_34 = df_34.rename(columns={'straftat': 'crimes_on_rank_4',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_4_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_4'})
df_34 = df_34.drop(columns='rank')

df_35 = df_34.merge(df_rank_5, on=['bundesland', 'sexus', 'year'], how='left')
df_35 = df_35.rename(columns={'straftat': 'crimes_on_rank_5',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_5_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_5'})
df_35 = df_35.drop(columns='rank')

df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5 = df_35.copy() 
df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.head()

,bundesland,sexus,year,offenders,offenders_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,Baden-Württemberg,M,2018,20122,9909,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),35.63,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.18,*26*00,Ladendiebstahl insgesamt,7.58,674000,Sachbeschädigung §§ 303-305a StGB,6.33,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,2.43
1,Bayern,M,2018,24065,10521,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),32.42,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.37,674000,Sachbeschädigung §§ 303-305a StGB,7.39,*26*00,Ladendiebstahl insgesamt,5.22,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,2.40
2,Berlin,M,2018,7941,15860,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.03,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),20.07,*26*00,Ladendiebstahl insgesamt,12.76,674000,Sachbeschädigung §§ 303-305a StGB,8.17,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",4.04
3,Brandenburg,M,2018,3910,12308,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),27.83,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.66,674000,Sachbeschädigung §§ 303-305a StGB,11.33,*26*00,Ladendiebstahl insgesamt,8.01,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.48
4,Bremen,M,2018,1934,15209,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.96,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),21.10,*26*00,Ladendiebstahl insgesamt,11.07,674000,Sachbeschädigung §§ 303-305a StGB,5.48,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",5.07


### Calculate Growth Rate

In [28]:
df_51 = df_laender_abs.copy()
df_51 = df_51[["schluessel", "straftat", "bundesland", "year", 'sexus', "heranwachsende_18_bis_unter_21"]]
df_51 = df_51[df_51["schluessel"] == "------"]
df_51 = df_51[["bundesland", "year", 'sexus', "heranwachsende_18_bis_unter_21"]]

df_54 = pd.DataFrame(columns = ["bundesland", "year", 'sexus', "heranwachsende_18_bis_unter_21", "growth_rate_abs"])

gender = ['X', 'W', 'M']
bundeslaender = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
for g in gender:
    df_52 = df_51[df_51["sexus"] == g]
    for b in bundeslaender:
        df_53 = df_52[df_52["bundesland"] == b]
        df_53["growth_rate_abs"] = df_53['heranwachsende_18_bis_unter_21'].pct_change() * 100
        df_54 = pd.concat([df_54, df_53])

df_54 = df_54.drop(columns='heranwachsende_18_bis_unter_21')

df_laender_growth_rate_abs = df_54.copy()
df_laender_growth_rate_abs.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_12634/1162673579.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_53["growth_rate_abs"] = df_53['heranwachsende_18_bis_unter_21'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_12634/1162673579.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_53["growth_rate_abs"] = df_53['heranwachsende_18_bis_unter_21'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_12634/1162673579.py:17:

,bundesland,year,sexus,growth_rate_abs
32,Baden-Württemberg,2018,X,NaN
560,Baden-Württemberg,2019,X,-6.087306
1088,Baden-Württemberg,2020,X,-9.793505
1616,Baden-Württemberg,2021,X,-12.367021
2144,Baden-Württemberg,2022,X,3.972469


In [29]:
df_61 = df_laender_rel.copy()
df_61 = df_61[["schluessel", "straftat", "bundesland", "year", 'sexus', "heranwachsende_18_bis_unter_21_rel"]]
df_61 = df_61[df_61["schluessel"] == "------"]
df_61 = df_61[["bundesland", "year", 'sexus', "heranwachsende_18_bis_unter_21_rel"]]

df_64 = pd.DataFrame(columns = ["bundesland", "year", 'sexus', "heranwachsende_18_bis_unter_21_rel", "growth_rate_rel"])

gender = ['X', 'W', 'M']
bundeslaender = ['Baden-Württemberg', 'Bayern', 'Berlin', 'Brandenburg', 'Bremen',
       'Hamburg', 'Hessen', 'Mecklenburg-Vorpommern', 'Niedersachsen',
       'Nordrhein-Westfalen', 'Rheinland-Pfalz', 'Saarland', 'Sachsen',
       'Sachsen-Anhalt', 'Schleswig-Holstein', 'Thüringen']
for g in gender:
    df_62 = df_61[df_61["sexus"] == g]
    for b in bundeslaender:
        df_63 = df_62[df_62["bundesland"] == b]
        df_63["growth_rate_rel"] = df_63['heranwachsende_18_bis_unter_21_rel'].pct_change() * 100
        df_64 = pd.concat([df_64, df_63])
        
df_64 = df_64.drop(columns='heranwachsende_18_bis_unter_21_rel')

df_laender_growth_rate_rel = df_64.copy()
df_laender_growth_rate_rel.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_12634/14226440.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_63["growth_rate_rel"] = df_63['heranwachsende_18_bis_unter_21_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_12634/14226440.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_63["growth_rate_rel"] = df_63['heranwachsende_18_bis_unter_21_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_12634/14226440.py:1

,bundesland,year,sexus,growth_rate_rel
32,Baden-Württemberg,2018,X,NaN
560,Baden-Württemberg,2019,X,-3.541884
1088,Baden-Württemberg,2020,X,-6.947527
1616,Baden-Württemberg,2021,X,-8.999620
2144,Baden-Württemberg,2022,X,6.479798


In [30]:
df_growth_rate_18_to_under_21 = df_laender_growth_rate_abs.merge(df_laender_growth_rate_rel, on=['bundesland', 'year', 'sexus'], how='left')
df_growth_rate_18_to_under_21.head()

,bundesland,year,sexus,growth_rate_abs,growth_rate_rel
0,Baden-Württemberg,2018,X,NaN,NaN
1,Baden-Württemberg,2019,X,-6.087306,-3.541884
2,Baden-Württemberg,2020,X,-9.793505,-6.947527
3,Baden-Württemberg,2021,X,-12.367021,-8.999620
4,Baden-Württemberg,2022,X,3.972469,6.479798


In [31]:
values_growth_rate_abs = df_growth_rate_18_to_under_21["growth_rate_abs"].tolist()
new_values_abs = []
for v in values_growth_rate_abs:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_abs.append(new_v)
df_growth_rate_18_to_under_21["growth_rate_abs"] = new_values_abs

values_growth_rate_rel = df_growth_rate_18_to_under_21["growth_rate_rel"].tolist()
new_values_rel = []
for v in values_growth_rate_rel:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_rel.append(new_v)
df_growth_rate_18_to_under_21["growth_rate_rel"] = new_values_rel

In [32]:
df_growth_rate_18_to_under_21.head()

,bundesland,year,sexus,growth_rate_abs,growth_rate_rel
0,Baden-Württemberg,2018,X,NaN,NaN
1,Baden-Württemberg,2019,X,-6.08,-3.53
2,Baden-Württemberg,2020,X,-9.78,-6.94
3,Baden-Württemberg,2021,X,-12.36,-8.99
4,Baden-Württemberg,2022,X,3.97,6.48


In [33]:
df_overview_fed_states_18_to_under_21 = df_laender_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.merge(df_growth_rate_18_to_under_21, on=['bundesland', 'sexus', 'year'], how='left')
#df_overview_fed_states = df_overview_fed_states.drop(columns='sexus')

df_overview_fed_states_18_to_under_21['age_group'] = 'heranwachsende_18_bis_unter_21'

# bring in correct order
df_overview_fed_states_18_to_under_21 = df_overview_fed_states_18_to_under_21[['bundesland', 'year', 'sexus', 'age_group', 'offenders', 'offenders_rel',
       'growth_rate_abs', 'growth_rate_rel',
       'schluessel_crimes_on_rank_1', 'crimes_on_rank_1',
       'percentage_of_rank_1_on_crime_total', 'schluessel_crimes_on_rank_2',
       'crimes_on_rank_2', 'percentage_of_rank_2_on_crime_total',
       'schluessel_crimes_on_rank_3', 'crimes_on_rank_3',
       'percentage_of_rank_3_on_crime_total', 'schluessel_crimes_on_rank_4',
       'crimes_on_rank_4', 'percentage_of_rank_4_on_crime_total',
       'schluessel_crimes_on_rank_5', 'crimes_on_rank_5',
       'percentage_of_rank_5_on_crime_total']]

df_overview_fed_states_18_to_under_21.tail(5)

,bundesland,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
235,Saarland,2022,X,heranwachsende_18_bis_unter_21,2061,7792,12.19,16.21,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.61,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),13.44,*26*00,Ladendiebstahl insgesamt,11.64,674000,Sachbeschädigung §§ 303-305a StGB,7.18,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.64
236,Sachsen,2022,X,heranwachsende_18_bis_unter_21,8836,8542,20.10,20.67,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),18.11,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",14.18,*26*00,Ladendiebstahl insgesamt,8.28,674000,Sachbeschädigung §§ 303-305a StGB,7.98,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.02
237,Sachsen-Anhalt,2022,X,heranwachsende_18_bis_unter_21,4688,8833,2.58,4.98,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),22.21,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.52,674000,Sachbeschädigung §§ 303-305a StGB,9.60,*26*00,Ladendiebstahl insgesamt,8.04,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.58
238,Schleswig-Holstein,2022,X,heranwachsende_18_bis_unter_21,5357,6243,1.32,5.21,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),25.91,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.49,*26*00,Ladendiebstahl insgesamt,10.64,674000,Sachbeschädigung §§ 303-305a StGB,7.28,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.68
239,Thüringen,2022,X,heranwachsende_18_bis_unter_21,4164,7867,1.41,1.94,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),26.06,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.70,*26*00,Ladendiebstahl insgesamt,9.39,674000,Sachbeschädigung §§ 303-305a StGB,9.29,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.54


In [34]:
df_overview_fed_states_18_to_under_21 = df_overview_fed_states_18_to_under_21.fillna('n.a.')

In [35]:
df_overview_fed_states_18_to_under_21.head()

,bundesland,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,Baden-Württemberg,2018,M,heranwachsende_18_bis_unter_21,20122,9909,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),35.63,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.18,*26*00,Ladendiebstahl insgesamt,7.58,674000,Sachbeschädigung §§ 303-305a StGB,6.33,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,2.43
1,Bayern,2018,M,heranwachsende_18_bis_unter_21,24065,10521,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),32.42,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.37,674000,Sachbeschädigung §§ 303-305a StGB,7.39,*26*00,Ladendiebstahl insgesamt,5.22,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,2.40
2,Berlin,2018,M,heranwachsende_18_bis_unter_21,7941,15860,n.a.,n.a.,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",21.03,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),20.07,*26*00,Ladendiebstahl insgesamt,12.76,674000,Sachbeschädigung §§ 303-305a StGB,8.17,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",4.04
3,Brandenburg,2018,M,heranwachsende_18_bis_unter_21,3910,12308,n.a.,n.a.,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),27.83,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.66,674000,Sachbeschädigung §§ 303-305a StGB,11.33,*26*00,Ladendiebstahl insgesamt,8.01,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,3.48
4,Bremen,2018,M,heranwachsende_18_bis_unter_21,1934,15209,n.a.,n.a.,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.96,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),21.10,*26*00,Ladendiebstahl insgesamt,11.07,674000,Sachbeschädigung §§ 303-305a StGB,5.48,210000,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",5.07


# Now Create same DataFrame for other age groups and then concat them to df_overview_fed_states

In [36]:
%store df_overview_fed_states_18_to_under_21

Stored 'df_overview_fed_states_18_to_under_21' (DataFrame)
